In [1]:
import pandas as pd
import os
import re
import csv
import numpy as np

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
!pip install datasets
!pip install rouge_score
# !pip install transformers
!pip install transformers -Uqq
!pip install datasets -Uqq
!pip install bert-score -Uqq
!pip install sacremoses
!pip install ohmeow-blurr -Uqq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import datasets
import pandas as pd
from fastai.text.all import *
from transformers import *

from blurr.text.data.all import *
from blurr.text.modeling.all import *

# import nltk
# nltk.download('punkt', quiet=True)

In [4]:
def clean(line):
    # line = line.strip().replace("newline_char", "||||||||||||||||||||||||||||")
    # line = line.strip().replace("newline_char", " ")
    line = line.strip()
    line = line.replace("( opens in new window )", "")
    line = line.replace("click to email this to a friend", "")
    line = line.replace("lick to share on whatsapp", "")
    line = line.replace("click to share on facebook", "")
    line = line.replace("share on facebook", "")
    line = line.replace("click to share on twitter", "")
    line = line.replace("click to share on pinterest", "")
    line = line.replace("click to share on tumblr", "")
    line = line.replace("click to share on google+", "")
    line = line.replace("feel free to share these resources in your social "
                        "media networks , websites and other platforms", "")
    line = line.replace("share share tweet link", "")
    line = line.replace("e-mail article print share", "")
    line = line.replace("read or share this story :", "")
    line = line.replace("share the map view in e-mail by clicking the share "
                        "button and copying the link url .     embed the map "
                        "on your website or blog by getting a snippet of html "
                        "code from the share button .     if you wish to "
                        "provide feedback or comments on the map , or if "
                        "you are aware of map layers or other "
                        "datasets that you would like to see included on our maps , "
                        "please submit them for our evaluation using this this form .", "")
    line = line.replace("share this article share tweet post email", "")
    line = line.replace("skip in skip x embed x share close", "")
    line = line.replace("share tweet pin email", "")
    line = line.replace("share on twitter", "")
    line = line.replace("feel free to weigh-in yourself , via"
                        "the comments section . and while you ’ "
                        "re here , why don ’ t you sign up to "
                        "follow us on twitter us on twitter .", "")
    line = line.replace("follow us on facebook , twitter , instagram and youtube", "")
    line = line.replace("follow us on twitter", "")
    line = line.replace("follow us on facebook", "")
    line = line.replace("play facebook twitter google plus embed", "")
    line = line.replace("play facebook twitter embed", "")
    line = line.replace("enlarge icon pinterest icon close icon", "")
    line = line.replace("follow on twitter", "")
    line = line.replace("autoplay autoplay copy this code to your website or blog", "")
    return line

In [5]:
def clean_str(s):
    s = s.lower()
    s = s.strip()
    s = s.replace('<unk>','')
    s = s.replace('`', '')
    # s = s.replace('.', '\n')
    s = s.replace(',', '')
    s = s.replace(';', '')
    s = s.replace('\'', '')
    s = s.replace('\"', '')
    s = s.replace('(', '')
    s = s.replace(')', '')
    s = s.replace('-', ' ')
    s = s.replace('<p>', '')
    s = s.replace('</p>', '')
    s = s.replace('<t>', '')
    s = s.replace('</t>', '')
    s = s.replace('[!@#$]', '')
    s = s.replace('– ', '')
    s = s.replace('  ', ' ')
    return s

In [6]:
def preprocess(s, stw, lem):
  words = nltk.word_tokenize(s)
  return " ".join([lem.lemmatize(word) for word in words if word not in stw])

In [10]:
# download train.src
!gdown 1ScGIcY_TcaIY57q9xdCQmi2EVAbmjHed 

Downloading...
From: https://drive.google.com/uc?id=1ScGIcY_TcaIY57q9xdCQmi2EVAbmjHed
To: /content/train.src
100% 550M/550M [00:03<00:00, 163MB/s]


In [9]:
# download train.tgt
!gdown 1iP2yUNI4-wkJvCMl-RsKBK9rJkEfnZdM

Downloading...
From: https://drive.google.com/uc?id=1iP2yUNI4-wkJvCMl-RsKBK9rJkEfnZdM
To: /content/train.tgt
100% 58.8M/58.8M [00:00<00:00, 145MB/s]


In [7]:
# download test.src
!gdown 1_eixB51Mrgyttq-Ke4hI5czcggTBhvXf

Downloading...
From: https://drive.google.com/uc?id=1_eixB51Mrgyttq-Ke4hI5czcggTBhvXf
To: /content/test.src
100% 69.4M/69.4M [00:00<00:00, 174MB/s]


In [8]:
# download test.tgt
!gdown 1Y56DFe5FROz9ikkYtEEAGcPBevV1IZa6

Downloading...
From: https://drive.google.com/uc?id=1Y56DFe5FROz9ikkYtEEAGcPBevV1IZa6
To: /content/test.tgt
100% 7.31M/7.31M [00:00<00:00, 205MB/s]


In [13]:
# list of dictionary
# each topic is dict that has following keys: 
# 'articles': list[str]
# 'paragraphs': list[list[str]] // each article is parsed into paragraphs
# 'summary': str
topics = []
src = 'test'

In [14]:
#get article
with open('./' + src + '.src') as file:
    raw = file.read()
    raw = clean(raw)
    
    topics = [{'articles': list(filter(str.strip, clean_str(articles).split('|||||')))} for articles in raw.split('\n')] # topics = [topic1, topic2, topic3, ...]
    
    print('total topics: ', len(topics)) #train / val - 5622 topics
    print('first topic has acticles: ', len(topics[0]["articles"]))
    
    for topic in topics:
        topic['paragraphs'] = [list(filter(str.strip, article.split("newline_char"))) for article in topic["articles"]]
        topic['articles'] = [article.replace("newline_char", '. ') for article in topic['articles']]
    file.close()

total topics:  5622
first topic has acticles:  2


In [15]:
#get summary
with open('./' + src + '.tgt') as file:
    raw = file.read()
    raw = clean_str(raw)
    
    summaries = raw.split('\n')
    
    for topic, summary in zip(topics, summaries):
        topic['summary'] = summary
    
    file.close()

In [ ]:
import pprint as pp
pp.pprint(topics[0])

In [16]:
from datasets import load_metric
metric = load_metric("rouge")

def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
'''
Summarizes corpus with Bart.
:parameter    
   :param corpus: list - dtf["text"]    
   :param max_len: length of the summary
:return    
    list of summaries
'''
import transformers

def bart(corpus, max_len):    
    nlp = transformers.pipeline("summarization")    
    lst_summaries = [nlp(txt, max_length=max_len)[0]["summary_text"].replace(" .", ".") for txt in corpus]    
    return lst_summaries

# from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from typing import List

def old_summarization_pipeline(text: List[str]) -> List[str]:
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    input_ids = tokenizer.batch_encode_plus(text, return_tensors='pt', max_length=1024, truncation=True, padding=True)['input_ids']
    summary_ids = model.generate(input_ids, max_new_tokens=256)
    summaries = [tokenizer.decode(s) for s in summary_ids]
    return summaries

In [62]:
articles_clean = []
references_clean = []

sws = nltk.corpus.stopwords.words('english')
lem = nltk.stem.wordnet.WordNetLemmatizer()

for i, topic in enumerate(topics):
  paragraphs = topic['paragraphs'][0]
  if(len(paragraphs)<3):
    continue

  # if i%500==0:
  #   print(i)

  # summary = preprocess(topic['summary'], sws, lem)
  # article = preprocess(topic['articles'][0], sws, lem)

  summary = topic['summary']
  article = topic['articles'][0]


  if len(summary) < (0.75*len(article)): #and 
    articles_clean.append(article)
    references_clean.append(summary)

print(len(articles_clean))

4000


In [ ]:
# baseline model - Bart
# Bert:
# seq2seq bidirectional encoder + GPT left-to-right autoregreesive decoder


# candidates = old_summarization_pipeline(articles_clean)
# print(len(candidates))

# print(calc_rouge_scores(candidates, references_clean))

9
{'rouge1': 22.7, 'rouge2': 7.8, 'rougeL': 13.6, 'rougeLsum': 13.5}


In [ ]:
n = 1
print(articles_clean[n])
print(candidates[n])
print(references_clean[n])

golden state latest version great secession . . . . . fed sacramento regulation southern california political sway resident one rural northern california county taking step leave state . . . siskiyou county board supervisor voted 4 1 tuesday pursue seceding california redding record searchlight reported . proponent say siskiyou form new state called jefferson county northern california southern oregon believe share similar interest . . . tuesday 100 people filled supervisor chamber many indicated support declaration searchlight reported . speaker asked audience favor nearly every hand room raised newspaper said . . . many proposed law unconstitutional deny u god given right said happy camp resident gabe garrison . need state make law fit way life . . . state jefferson place want raise son kayla brown said . . . resident complaint include lack representation sacramento insufficient attention major issue county water right rural fire prevention fee searchlight reported . . . government t

In [65]:
all_data = [articles_clean, references_clean]
print(len(articles_clean))

4000


In [66]:
#Import the pretrained model
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, model_cls=BartForConditionalGeneration)

In [67]:
text_gen_kwargs = default_text_gen_kwargs(hf_config, hf_model, task='summarization'); text_gen_kwargs

{'max_length': 142,
 'min_length': 56,
 'do_sample': False,
 'early_stopping': True,
 'num_beams': 4,
 'temperature': 1.0,
 'top_k': 50,
 'top_p': 1.0,
 'typical_p': 1.0,
 'repetition_penalty': 1.0,
 'bad_words_ids': None,
 'bos_token_id': 0,
 'pad_token_id': 1,
 'eos_token_id': 2,
 'length_penalty': 2.0,
 'no_repeat_ngram_size': 3,
 'encoder_no_repeat_ngram_size': 0,
 'num_return_sequences': 1,
 'decoder_start_token_id': 2,
 'use_cache': True,
 'num_beam_groups': 1,
 'diversity_penalty': 0.0,
 'output_attentions': False,
 'output_hidden_states': False,
 'output_scores': False,
 'return_dict_in_generate': False,
 'forced_bos_token_id': 0,
 'forced_eos_token_id': 2,
 'remove_invalid_values': False,
 'exponential_decay_length_penalty': None,
 'suppress_tokens': None,
 'begin_suppress_tokens': None}

In [68]:
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(
    hf_arch, hf_config, hf_tokenizer, hf_model, max_length=256, max_tgt_length=130, text_gen_kwargs=text_gen_kwargs
)

blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('article'), get_y=ColReader('summary'), splitter=RandomSplitter())


In [69]:
df = pd.DataFrame(all_data, index=['article', 'summary']).T
df.head()

,article,summary
0,gop eyes gains as voters in 11 states pick governors . . enlarge this image toggle caption jim cole/ap jim cole/ap . . voters in 11 states will pick their governors tonight and republicans appear on track to increase their numbers by at least one with the potential to extend their hold to more than two thirds of the nations top state offices. . . eight of the gubernatorial seats up for grabs are now held by democrats three are in republican hands. republicans currently hold 29 governorships democrats have 20 and rhode islands gov. lincoln chafee is an independent. . . polls and rac...,its a race for the governors mansion in 11 states today and the gop could end the night at the helm of more than two thirds of the 50 states. the gop currently controls 29 of the countrys top state offices its expected to keep the three republican ones that are up for grabs utah north dakota and indiana and wrest north carolina from the dems. that brings its toll to 30 with the potential to take three more reports npr. races in montana new hampshire and washington are still too close to call and in all three democrat incumbents arent seeking reelection. the results could have a big impact ...
1,. . . . update: 4/19/2001 read richard metzger: how i a married middle aged man became an accidental spokesperson for gay rights overnight on boing boing . . it’s time to clarify a few details about the controversial “hey facebook what’s so wrong with a pic of two men kissing?” story as it now beginning to be reported in the mainstream media and not always correctly. . . first of all with regards to the picture: . . the photo which was used to illustrate my first post about the john snow kiss in is a promotional still from the british soap opera “eastenders.” it features one of t...,it turns out facebook is only guilty of about half of what it’s been accused of in the gay kiss incident. the social networking site apologized yesterday for taking down an image used to promote a “kiss in” event in london. “the photo in question does not violate our statement of rights and responsibilities and was removed in error” the site said in a statement according to the advocate. but facebook did not as has been reported in several places take down the kiss in event itself. here’s what happened: the photo facebook took down was posted by the dangerous minds blog to promote the even...
2,its the golden states latest version of the great secession. . . . . fed up by sacramentos regulations and southern californias political sway residents in one rural northern california county are taking steps to leave the state. . . the siskiyou county board of supervisors voted 4 1 on tuesday to pursue seceding from california the redding record searchlight reported. proponents say siskiyou should form a new state called jefferson with other counties in northern california and southern oregon they believe share similar interests. . . on tuesday more than 100 people filled the s...,not a big fan of southern california? neither is northern california apparently. supervisors in rural siskiyou county voted 4 1 tuesday in favor of seceding from the state reports the times standard. the county thinks state officials in sacramento are too focused on the big metro areas of the south and thus want to form a state of its own—to be called jefferson. supporters want to invite other northern counties and perhaps a few from oregon to join too notes the los angeles times. ”we have to have government thats local understands our issues and has empathy” says one local rancher. its cl...
3,after a year in which liberals scored impressive high profile supreme court victories conservatives could be in line for wins on some of this terms most contentious issues as the justices consider cases that could gut public sector labor unions and roll back affirmative action at state universities. . . however as the courts new term kicks off monday uncertainty surrounds several other politically po

In [70]:
dls = dblock.dataloaders(df, batch_size = 2)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


In [71]:
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'en' },
            'returns': ["precision", "recall", "f1"]
        }
    }

In [72]:
#Model
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

In [73]:
learn = Learner(dls, 
                model,
                opt_func=ranger,
                loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,
                splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()
learn.create_opt() 
learn.freeze()

In [74]:
learn.lr_find()

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "
/usr/local/lib/python3.7

In [ ]:
b = dls.one_batch()
preds = learn.model(b[0])
len(preds),preds[0], preds[1].shape

In [ ]:
learn.fit_one_cycle(1, lr_max=3e-5, cbs=fit_cbs)

In [ ]:
learn.show_results(learner=learn, max_n=2)

In [ ]:
test_article = """
The past 12 months have been the worst for aviation fatalities so far this decade - with the total of number of people killed if airline 
crashes reaching 1,050 even before the Air Asia plane vanished. Two incidents involving Malaysia Airlines planes - one over eastern Ukraine and the other in the Indian Ocean - led to the deaths of 537 people, while an Air Algerie crash in Mali killed 116 and TransAsia Airways crash in Taiwan killed a further 49 people. The remaining 456 fatalities were largely in incidents involving small commercial planes or private aircraft operating on behalf of companies, governments or organisations. Despite 2014 having the highest number of fatalities so far this decade, the total number of crashes was in fact the lowest since the first commercial jet airliner took off in 1949 - totalling just 111 across the whole world over the past 12 months. The all-time deadliest year for aviation was 1972 when a staggering 2,429 people were killed in a total of 55 plane crashes - including the crash of Aeroflot Flight 217, which killed 174 people in Russia, and Convair 990 Coronado, which claimed 155 lives in Spain. However this year's total death count of 1,212, including those presumed dead on board the missing Air Asia flight, marks a significant rise on the very low 265 fatalities in 2013 - which led to it being named the safest year in aviation since the end of the Second World War. Scroll down for videos. Deadly: The past 12 months have been the worst for aviation fatalities so far this decade - with the total of number of people killed if airline crashes reaching 1,158 even before the Air Asia plane (pictured) vanished. Fatal: Two incidents involving Malaysia Airlines planes - one over eastern Ukraine (pictured) and the other in the Indian Ocean - led to the deaths of 537 people. Surprising: Despite 2014 having the highest number of fatalities so far this decade, the total number of crashes was in fact the lowest since the first commercial jet airliner took off in 1949. 2014 has been a horrific year for Malaysia-based airlines, with 537 people dying on Malaysia Airlines planes, and a further 162 people missing and feared dead in this week's Air Asia incident. In total more than half the people killed in aviation incidents this year had been flying on board Malaysia-registered planes. In January a total of 12 people lost their lives in five separate incidents, while the same number of crashes in February killed 107. 
"""

In [ ]:
outputs = learn.blurr_summarize(test_article, early_stopping=True, num_beams=4, num_return_sequences=3)

for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')